### Import Required Packages

In [1]:
import os
from huggingface_hub import InferenceClient
import pandas as pd

/Users/vaisiyabalakrishnan/techjam/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Initialize the Inference Client

In [2]:
client = InferenceClient(
    provider="nscale",
    api_key=os.environ["HF_TOKEN"],
)

### Few-shot Examples

In [3]:
FEW_SHOT_EXAMPLES = """
You are a system that classifies Google location reviews into one of four categories:
- Ad: Promotional or advertisement content.
- Rant: Angry or exaggerated complaints, often with excessive punctuation or all-caps.
- Irrelevant: Not related to the location being reviewed.
- Valid: A genuine and relevant review about the location.

Examples:
Review: "Best pizza in town! Fresh ingredients and great service."
Label: Valid

Review: "BUY ONE GET ONE FREE! Come to my shop now, limited offer!"
Label: Ad

Review: "THIS PLACE IS THE WORST!!! NEVER COMING BACK. HORRIBLE SERVICE!!!!!"
Label: Rant

Review: "I think the government is doing a terrible job with taxes."
Label: Irrelevant
"""

### Batch Classification Prompt

In [4]:
def create_batch_prompt(reviews):
    reviews_list = "\n".join(
        [f"Review {i+1}: {r}" for i, r in enumerate(reviews)]
    )
    return f"""{FEW_SHOT_EXAMPLES}
    Now classify the following reviews:
    {reviews_list}

    Output format:
    Review 1: <Label>
    Review 2: <Label>
    ...
    """

### Classify Reviews in Batches

In [5]:
def classify_batch(reviews):
    prompt = create_batch_prompt(reviews)
    response = client.chat.completions.create(
        model="Qwen/Qwen3-4B-Instruct-2507",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message["content"]

### Label Combined Dataset

In [6]:
def label_reviews(df, text_col="review_text", batch_size=10, max_reviews=100):
    labeled = []
    reviews = df[text_col].astype(str).tolist()[:max_reviews]
    
    for i in range(0, len(reviews), batch_size):
        batch = reviews[i:i+batch_size]
        output = classify_batch(batch)

        # Parse output: expects "Review 1: Valid" style
        for line in output.splitlines():
            if line.strip() and line.startswith("Review"):
                try:
                    idx, label = line.split(":", 1)
                    labeled.append(label.strip())
                except:
                    labeled.append("Unknown")
        
        print(f"Processed {i+len(batch)} / {len(reviews)}")
    
    df = df.iloc[:max_reviews].copy()
    df["label"] = labeled
    return df

### Label Some of Cleaned Combined Data

In [7]:
data_path = "../data/clean/cleaned_combined_reviews.csv"
df = pd.read_csv(data_path)
cleaned_df = df.copy()
labeled_df = label_reviews(df, batch_size=10, max_reviews=50)
labeled_df.to_csv("../data/label/labeled_combined_reviews.csv", index=False)

Processed 10 / 50
Processed 20 / 50
Processed 30 / 50
Processed 40 / 50
Processed 50 / 50
